# Mitigating bias in recommender systems

This is an introduction to fairness in recommender systems. A recommender system aims to recommend the best item according to the user preference. In this tutorial, we will focus on the task of correctly predicting users' music preference.

A recommender system can be biased in multiple ways. For example, we may be concerned that the artists in our database will not get equal representation (item fairness). Alternative, our main concern may be that different groups of users (e.g. male/female users) will get different music recommendations (user fairness). In the following, we will show how to mitigate item fairness.

### 0 - Importing modules and loading the data

We will start by importing the example dataset, which we host on our library. The [datatset](https://www.kaggle.com/datasets/ravichaubey1506/lastfm) contains a set of artists that were downloaded by users. It includes personal information about the user, specifically sex and country of origin. A user can download more than one artist.

In [1]:
# sys path
import sys
sys.path.append('../../')

In [2]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
from holisticai.datasets import load_last_fm

bunch = load_last_fm()
lastfm = bunch['frame']
lastfm['score'] = 1
lastfm

,user,artist,sex,country,score
0,1.0,red hot chili peppers,f,Germany,1
1,1.0,the black dahlia murder,f,Germany,1
2,1.0,goldfrapp,f,Germany,1
3,1.0,dropkick murphys,f,Germany,1
4,1.0,le tigre,f,Germany,1
...,...,...,...,...,...
289950,19718.0,bob dylan,f,Canada,1
289951,19718.0,pixies,f,Canada,1
289952,19718.0,the clash,f,Canada,1
289953,19718.0,a tribe called quest,f,Canada,1


In [3]:
# HELPER TOOLS -- NOTHING TO DO

def items_liked_by_user(data_matrix, u):
    return np.nonzero(data_matrix[u])[0]

def explode(arr, num_items):
    out = np.zeros(num_items)
    out[arr] = 1
    return out

def recommended_items(model, data_matrix, k):
    recommended_items_mask = data_matrix>0
    candidate_index = ~recommended_items_mask
    candidate_rating = model.pred*candidate_index
    return np.argsort(-candidate_rating,axis=1)[:,:k]

def recommender_rmse(mat_pred, mat_true):
    mask = mat_true>0
    rmse = np.sqrt(np.sum(np.power(mat_pred-mat_true,2)*mask)/np.sum(mask))
    return rmse

def recommender_mae(mat_pred, mat_true):
    mask = mat_true>0
    mae = np.sum(np.abs(mat_pred-mat_true)*mask)/np.sum(mask)
    return mae

### 2 - Pre-processing the data and training a model

In [4]:
# Imports
from holisticai.bias.metrics import recommender_bias_metrics
from holisticai.utils import recommender_formatter
from sklearn.metrics import mean_absolute_error

In [5]:
# Each interaction results in a non-nan entry in the dataframe.
df_pivot, p_attr = recommender_formatter(lastfm, users_col='user', groups_col='sex', items_col='artist', scores_col='score', aggfunc='mean')
data_matrix = df_pivot.fillna(0).to_numpy()

Train a Non Negative Matrix Factorization Model

In [6]:
from holisticai.bias.mitigation import NonNegativeMF

mf = NonNegativeMF(K=40)
mf.fit(data_matrix)

new_items = recommended_items(mf, data_matrix, 10)
new_recs = [explode(new_items[u], len(df_pivot.columns)) for u in range(df_pivot.shape[0])]
new_df_pivot_db = pd.DataFrame(new_recs, columns = df_pivot.columns)

mat = new_df_pivot_db.replace(0,np.nan).to_numpy()

/Users/giuliofilippi/opt/anaconda3/envs/torch-nightly/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1692: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [7]:
# Efficacy metric
recommender_mae(mf.pred, data_matrix)

0.7505365813425107

In [8]:
# Bias metrics
recommender_bias_metrics(mat_pred=mat, metric_type='item_based')

/Users/giuliofilippi/Documents/GitHub/holisticai/tutorials/mitigating_bias_tutorials/../../holisticai/utils/_recommender_tools.py:228: RuntimeWarning: divide by zero encountered in log
  return -np.sum(np.where(p != 0, p * np.log(p), 0))
/Users/giuliofilippi/Documents/GitHub/holisticai/tutorials/mitigating_bias_tutorials/../../holisticai/utils/_recommender_tools.py:228: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(np.where(p != 0, p * np.log(p), 0))


,Value,Reference
Metric,,
Aggregate Diversity,0.485060,1
GINI index,0.826159,0
Exposure Distribution Entropy,5.455747,-
Average Recommendation Popularity,792.536493,-


<font color='red'> **Question**
- **Comment on the values of the above metrics**
<font > 

### 3 - Mitigating bias
We will now show how we can mitigate bias using the holisticai library. More specifically we will focus on item fairness, and use Blind Spot Aware Matrix Factorization.

Reference:
        Sun, Wenlong, et al. "Debiasing the human-recommender system
        feedback loop in collaborative filtering." Companion Proceedings
        of The 2019 World Wide Web Conference. 2019.

In [9]:
# Imports
from holisticai.bias.mitigation import BlindSpotAwareMF

<font color='red'>  **Task 1**
- **Train a Blind Spot Aware Matrix Factorization Model** (parameters : K=40, beta=0.02, steps=10, alpha=0.002, lambda=0.008, verbose=1)
<font >

In [10]:
# TODO


<font color='red'>  **Task 2**
- **Evaluate your Model's efficacy**
<font >

In [11]:
# TODO


You should get the following results:

| Metric | Value | Reference |
| --- | --- | --- |
| RMSE | 0.217| 0 |
| MAE | 0.158 | 0 |




<font color='red'>  **Task 3**
- **Evaluate your Model's bias**
<font >

In [12]:
# TODO


You should get the following results:

| Metric | Value | Reference |
| --- | --- | --- |
| Aggregate Diversity | 0.72 | 1 |
| Gini Index   | 0.66 | 0 |
| Avg Recommendation Pop   | 414 | - |




<font color='red'> **Questions**
- **Has efficacy increased or decreased when training with bias mitigation?**
- **Has bias increased or decreased when training with bias mitigation?**
<font > 